In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

In [ ]:
# Yahoo LtR challenge
def parser(filename):
    with open(filename, 'r') as f:
        rel_scores = []
        qids = []
        features = {}
        for line in f:
            data = line.split()
            rel_scores.append(int(data[0]))
            qids.append(int(data[1].split(':')[1]))

            for pair in data[2:]:
                col, val = pair.split(':')
                col = int(col)
                val = float(val)

                if col not in features:
                    features[col] = [0.0] * (len(rel_scores)-1)
                features[col].append(val)
            
            for i in range(max(features.keys()) + 1):
                if i not in features:
                    features[i] = [0.0] * len(rel_scores)
                if len(features[i]) < len(rel_scores):
                    features[i].extend([0.0] * (len(rel_scores) - len(features[i])))
    
    features['score'] = rel_scores
    features['qid'] = qids

    return pd.DataFrame(data=features)

In [ ]:
yahoo_train = parser('../datasets/Yahoo/ltrc_yahoo/set1.train.txt')
yahoo_valid = parser('../datasets/Yahoo/ltrc_yahoo/set1.validation.txt')
yahoo_test = parser('../datasets/Yahoo/ltrc_yahoo/set1.test.txt')